In [1]:
from pyspark import SparkContext

def createIndex(shapefile):
    import rtree
    import fiona.crs
    import geopandas as gpd
    zones = gpd.read_file(shapefile).to_crs(fiona.crs.from_epsg(4326))
    index = rtree.Rtree()
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

def findZone(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        if zones.geometry[idx].contains(p):
            return idx

In [2]:
def getDrugsFromFileToSets(fn1,fn2):
    import re
    w=set()
    with open(fn1) as reader:
        rows = reader.read()
        for row in rows.split("\n"):
            w.add(" ".join(re.split(r'\W+',row)))
    with open(fn2) as reader:
        rows = reader.read()
        for row in rows.split("\n"):
            w.add(" ".join(re.split(r'\W+',row)))
    return w

In [3]:
def filterTweetWithoutDrugsKeyword(tweet,drugs_set):
    import re
    lst = re.split(r'\W+',tweet)
    lst = list(filter(lambda x:x!="",lst))
    i=0
    for word1 in lst:
        j=i+1
        temp =word1
        if(temp in drugs_set):
            return True
        for word2 in lst[j:]:
            temp = temp+" "+word2
            if(temp in drugs_set):
                return True
        i=i+1
    return False

In [4]:
def partition(tweetData):
    import pyproj
    import shapely.geometry as geom
    proj = pyproj.Proj(init="epsg:4326", preserve_units=False)
    main_index2 , main_zone2 = createIndex('500cities_tracts.geojson')
    drugs_set = broadcast_drug_set.value
    main_zone = broadcast_main_zone.value
#     main_index = broadcast_main_index.value
    main_index = main_index2
    city_index = None
    for tweet in tweetData:
        tweet_split = tweet.split('|')
        try:
            x = float(tweet_split[2])
            y = float(tweet_split[1])
            point = geom.Point((x,y))
            city_index = findZone(point,main_index,main_zone)
        except:
            pass
        if not city_index:
            pass
            #print(tweet_split[0])
        else:
            if (int(main_zone['plctrpop10'][city_index])!=0) and filterTweetWithoutDrugsKeyword(tweet_split[5],drugs_set):
                yield(main_zone['plctract10'][city_index],main_zone['plctrpop10'][city_index]),1.0

In [5]:
def normalizer(data):
    return data[0][0],float(data[1])/float(data[0][1])

In [6]:
if __name__=='__main__':
    #sc = SparkContext()
    main_index1 , main_zone1 = createIndex('500cities_tracts.geojson')
    broadcast_main_zone = sc.broadcast(main_zone1)
    broadcast_main_index = sc.broadcast(main_index1)
    broadcast_drug_set=sc.broadcast(getDrugsFromFileToSets('drug_illegal.txt','drug_sched2.txt'))
    sc.textFile('/data/share/bdm/tweets-100m.csv').mapPartitions(partition).reduceByKey(lambda x,y:x+y).map(normalizer).sortBy(lambda x:x[0]).saveAsTextFile("final006")

Py4JJavaError: An error occurred while calling o95.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/home/prajwal/Desktop/BigDataCourse/Homeworks/Homework6/final006 already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:287)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	a